In [ ]:
from convert import *
from function1 import *
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Dropout, GRU
from keras.optimizers import Adam, SGD, Adagrad
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [ ]:
film_sayisi = 3
window_size = 5
films = list(range(film_sayisi))

In [ ]:
# Loading the dictionary
with open("dictionary.txt","r",encoding="utf8") as file:
    dictionary = [string.replace("\n","") for string in file.readlines()]

In [ ]:
# Creating scripts[][]
scripts = []
for j in films:
    with open(f"{j}_enc.txt","r") as file:
        scripts.append(list(map(int,list(file.read().splitlines()))))

In [ ]:
# Turning int to [int]
def func(liste):
    return [[x] for x in liste]
scripts = list(map(func,scripts))

In [ ]:
# To categorical -> [0,0,0,...,1,...0,0,0]
for j in films:
    scripts[j] = to_categorical(scripts[j],num_classes=len(dictionary)+1,dtype="byte")

In [ ]:
# Creating inputs and outputs lists
inputs = []
outputs = []
for j in films:
    inputs.append(None)
    outputs.append(None)


# Creating sublists
for j in films:
    inputs[j],outputs[j] = create_sublists(scripts[j],window_size)


# Removing [films] layer in lists
inputs_temp = []
for j in films:
    for k in range(len(inputs[j])):
        inputs_temp.append(inputs[j][k])
inputs = inputs_temp
del inputs_temp
outputs_temp = []
for j in films:
    for k in range(len(outputs[j])):
        outputs_temp.append(outputs[j][k])
outputs = outputs_temp
del outputs_temp

In [ ]:
for i in range(3):
    print(f"{np.shape(inputs[i])} -> {np.shape(outputs[i])}")
    print(f"{inputs[i]} -> {outputs[i]}")

(5, 7338) -> (7338,)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] -> [0 0 0 ... 0 0 0]
(5, 7338) -> (7338,)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] -> [0 0 0 ... 0 0 0]
(5, 7338) -> (7338,)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] -> [0 0 0 ... 0 0 0]


In [ ]:
# Creating train data and test data, x is for input and y is for output
x_train, x_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.3, random_state=42)
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

In [ ]:
print(np.shape(x_train))
print(np.shape(y_train))
print(np.shape(x_test))
print(np.shape(y_test))

for i in range(2):
  print("")

print(len(x_train[0][0]))
print(y_train[0])

(38906, 5, 7338)
(38906, 7338)
(16675, 5, 7338)
(16675, 7338)


7338
[0 0 0 ... 0 0 0]


In [ ]:
# Creating model (ANN)
window_size = 5
model=Sequential([
    Flatten(input_shape=(window_size, len(x_train[0][0]), 1),dtype="int8"),
    Dense(len(x_train[0][0]), activation="relu"),
    Dense(len(x_train[0][0]), activation="softmax")
])

# Showing properties
model.summary()

# Setting the optimizer algorithm and compiling the model
opt = Adagrad(learning_rate = 0.01)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print("Model ready")

# Training the model
history = model.fit(x_train, y_train, epochs=10, batch_size=16)
# Saving the model
model.save("drive/MyDrive/Colab Notebooks/Dense_10epochs.h5")

In [ ]:
# Creating model (RNN)
model2 = Sequential()
model2.add(LSTM(units = 1000, input_shape = (window_size, len(x_train[0][0])), return_sequences = True))
model2.add(Dropout(0.2))
model2.add(LSTM(units = 1000, return_sequences = True))
model2.add(Dropout(0.2))
model2.add(LSTM(units = 1000))
model2.add(Dropout(0.2))
model2.add(Dense(units = len(y_train[0])))

# Showing properties
model2.summary()

# Setting the optimizer algorithm and compiling the model
opt = Adagrad(learning_rate = 0.01)
model2.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print("Model2 ready")

# Training the model
history2 = model2.fit(x_train, y_train, epochs=10, batch_size=16)
# Saving the model
model2.save("drive/MyDrive/Colab Notebooks/LSTM_3x1000_10epochs.h5")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 5, 1000)           33356000  
                                                                 
 dropout_3 (Dropout)         (None, 5, 1000)           0         
                                                                 
 lstm_4 (LSTM)               (None, 5, 1000)           8004000   
                                                                 
 dropout_4 (Dropout)         (None, 5, 1000)           0         
                                                                 
 lstm_5 (LSTM)               (None, 1000)              8004000   
                                                                 
 dropout_5 (Dropout)         (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 7338)             

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
